In [11]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [12]:
train_data_dir = r'..\..\spectro_fonctionnels\Train'
test_data_dir = r'..\..\spectro_fonctionnels\Test'
img_width, img_height = 336, 336

In [13]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32 
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 7642 images belonging to 2 classes.
Found 1910 images belonging to 2 classes.


In [14]:
# On définit l'architecture du modèle :

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# On compile le modeles :

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# On affiche le model :

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 334, 334, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 334, 334, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 167, 167, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 165, 165, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 165, 165, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 82, 82, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 80, 80, 64)        1

In [15]:
# On définit les parametres pour l'entrainement :
epochs = 20
train_samples = 7642 
test_samples = 1910 

In [16]:
historique = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=test_samples// batch_size)

Epoch 1/20
238/238 [==============================] - 1366s 6s/step - loss: 0.5504 - acc: 0.7313 - val_loss: 0.4850 - val_acc: 0.7696
Epoch 2/20
238/238 [==============================] - 691s 3s/step - loss: 0.4109 - acc: 0.8256 - val_loss: 0.3663 - val_acc: 0.8506
Epoch 3/20
238/238 [==============================] - 821s 3s/step - loss: 0.3695 - acc: 0.8469 - val_loss: 0.2892 - val_acc: 0.8824
Epoch 4/20
238/238 [==============================] - 609s 3s/step - loss: 0.3215 - acc: 0.8687 - val_loss: 0.2593 - val_acc: 0.8962
Epoch 5/20
238/238 [==============================] - 646s 3s/step - loss: 0.2731 - acc: 0.8987 - val_loss: 0.2328 - val_acc: 0.9068
Epoch 6/20
238/238 [==============================] - 870s 4s/step - loss: 0.2443 - acc: 0.9070 - val_loss: 0.2279 - val_acc: 0.9089
Epoch 7/20
238/238 [==============================] - 593s 2s/step - loss: 0.2278 - acc: 0.9123 - val_loss: 0.1887 - val_acc: 0.9296
Epoch 8/20
238/238 [==============================] - 421s 2s/step -

In [17]:
model.save(r'save/model_336')

In [9]:
model2 = load_model(r'save/premier_model_potable')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
epochs = range(1,20)
plt.plot(epochs, historique.history['accuracy'], label='Accuracy')
plt.plot(epochs, historique.history['val_accuracy'], label='Validation Accuracy')
plt.plot(epochs, historique.history['loss'], label='Loss')
plt.plot(epochs, historique.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation loss and accuracy for the heavy layer fully connected')
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Values')
plt.show()

NameError: name 'train_generator' is not defined